In [1]:
from tqdm.auto import tqdm 
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nf
from netCDF4 import Dataset
%matplotlib inline
import glob
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [2]:
datapath="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/Data/ECMWF/ERA5_25kmx3hr/"
path="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/"
output="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/"
target="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/targets/"

In [3]:
!ls

200km_timeseries_2020.ipynb  Outer2020_nio.ipynb
2018_nio.ipynb		     outer_2020_wp.ipynb
2019_inner_nio.ipynb	     outer_rad_timeseries.ipynb
2019_wp_inner.ipynb	     Percentile.ipynb
aavg.ipynb		     Pmin_test.ipynb
amphan_roc.ipynb	     precip_test.ipynb
confusion_roc.ipynb	     ROC_threshold.ipynb
ERA5_ts_2020_inner.ipynb     skillscores.ipynb
ERA5_ts_2020_outer.ipynb     spatial_avg_test.ipynb
Inner_2020_nio.ipynb	     test_igpp.ipynb
inner_wp_2020.ipynb	     test_kotal_skill.ipynb
mean_timeseries_final.ipynb  test_levs.ipynb
outer_2019_nio.ipynb	     timeseries_200_amphan.ipynb
outer_2019_wp.ipynb	     Untitled.ipynb


In [4]:
tracklist = sorted(glob.glob(path+'tracks_wp/*2020*'))

In [5]:
tracklist

['/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20200508.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20201003.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20201016.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20201021.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20201025.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20201029.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/nwp_20201108.csv']

#order 2020
vongfong
chanhom
saudel
molave
goni
atsani
vamco

In [38]:
maindf_vongfong=pd.read_csv(target+'2020_w10max_nwp_vongfong.csv',delimiter=r",")
maindf_vongfong.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_vongfong=maindf_vongfong.drop('a', axis=1)

In [39]:
extra=pd.read_csv(output+'nwp/inder_2020_nwp_vongfong.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

In [6]:
dm1=xr.open_mfdataset([datapath+'/vor/vor_2020.nc',datapath+'/rhum/rhum_2020.nc'])

In [67]:
extra.shape

(81, 22)

In [7]:
def largearea_withpres(dataset=None,invar=None,indices=None,tc_irad=None):
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","plev","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices))),
               plev=(["plev"],dataset.plev.data)),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),len(dataset.plev.data),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        for ip in range(len(dm1.plev.data)):
            try:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
            except:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out  

In [8]:
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out
###############################read year from data and track ####################################

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/svars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/*'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

####change year below##########################
dm2,tracklist,era5_date,era5_hour = readyear(2020) 

TCsl_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['u10','v10','2mdewtmp','2mtmp','conv_ppt','tot_cld_ice',\
           'tot_cldwtr','tot_cld_rain','vi_div_cld_froz_wtr','vi_div_cld_liq_wtr','vi_div_gpot_flux',\
           'vi_div_ke_flux','vi_div_mass_flux','vi_div_moisture_flux','vi_div_olr_flux','vi_div_tot_enrgy_flux',\
           'vi_ke','vi_pe_inte','vi_pe_ie_latentenrgy','vi_temp','vi_olr','vi_tot_enrgy','vi_moisture_div']
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCsl_ts.append(tsdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [10]:
##########radiation variables############
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out
###############################read year from data and track ####################################

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/radvars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/*'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2020)

TCrl_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['cape','inst_10m_wnd_gst','inst_moisture_flux','inst_ssh_flux','surfmean_swr_flux','surfmean_lhf',\
           'surfmean_lwr_flux','surfmean_shf','dwnwrdmean_swr_flux','topmean_lwr_flux','topmean_swr_flux',\
           'vimean_moisture_div','surf_lhf','surf_shf','tot_suprcool_liqwtr','tot_wtr_vpr']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCrl_ts.append(tsdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [11]:
##########precip, mslp, sst variables############
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out
###############################read year from data and track ####################################

def readyear(year=None):
    dm2 = xr.open_mfdataset([datapath+'/prate/prates_2020.nc',datapath+'/mslp/mslp_2020.nc',datapath+'/sst/sst_2020.nc'])
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_wp/*'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2020)

TCpr_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['mslp','conv_rrate','ls_rrate','mn_conv_prate','mn_ls_prate','mn_tot_prate','sst']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCpr_ts.append(tsdict)

  0%|          | 0/7 [00:00<?, ?it/s]

/tmp/ipykernel_2593181/3386996906.py:94: RuntimeWarning: Mean of empty slice
  tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]


In [25]:
rlev_vongfong=pd.DataFrame.from_dict(TCrl_ts[0])

In [46]:
prlev_vongfong=pd.DataFrame.from_dict(TCpr_ts[0])

In [47]:
slev_vongfong=pd.DataFrame.from_dict(TCsl_ts[0])

In [50]:
prlev_vongfong['mslp']=prlev_vongfong['mslp']/100

In [51]:
prlev_vongfong['mslp']

0     1006.904209
1     1007.775674
2     1009.236318
3     1009.583965
4     1007.478916
         ...     
76    1004.091035
77    1004.137275
78    1005.254336
79    1005.059487
80    1004.339019
Name: mslp, Length: 81, dtype: float64

In [52]:
ts1_vongfong=pd.concat([maindf_vongfong,extra,slev_vongfong,rlev_vongfong,prlev_vongfong], axis=1, join='inner')

In [53]:
ts1_vongfong

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,surf_shf,tot_suprcool_liqwtr,tot_wtr_vpr,mslp,conv_rrate,ls_rrate,mn_conv_prate,mn_ls_prate,mn_tot_prate,sst
0,10.309203,-0.000001,-0.000005,5.563559e-06,7.814137e-06,5.854761e-06,-6.114945e-06,336.610168,401.203491,401.203491,...,-50804.50,0.043295,62.715279,1006.904209,0.000035,0.000042,0.000035,0.000054,0.000088,302.785969
1,10.910505,0.000008,-0.000003,-3.635838e-06,-9.401367e-07,7.783531e-07,4.811327e-06,337.293274,400.864471,400.864471,...,-49018.25,0.035915,63.555801,1007.775674,0.000102,0.000036,0.000085,0.000034,0.000119,302.795067
2,9.938855,0.000017,0.000018,1.688236e-06,-3.315896e-06,-8.495769e-06,2.562160e-06,336.432892,399.526886,399.526886,...,-46483.25,0.046591,64.174829,1009.236318,0.000090,0.000034,0.000101,0.000027,0.000128,302.799229
3,8.958860,0.000019,0.000022,6.507719e-07,-4.473000e-06,5.714251e-07,6.604291e-06,335.481506,399.772156,399.772156,...,-44218.25,0.058461,64.336866,1009.583965,0.000201,0.000031,0.000171,0.000032,0.000203,302.813881
4,9.296708,0.000013,0.000016,-7.199931e-06,-6.999429e-06,6.906871e-06,7.085467e-06,336.512817,399.554321,399.554321,...,-53495.50,0.057103,63.386686,1007.478916,0.000212,0.000036,0.000171,0.000032,0.000204,302.820923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,12.185334,0.000024,0.000012,8.853050e-06,3.857888e-06,-1.038136e-05,-1.042617e-05,328.694977,402.119659,402.119659,...,-38848.25,0.017607,52.141068,1004.091035,0.000135,0.000068,0.000193,0.000064,0.000257,300.961965
77,12.407811,0.000016,-0.000004,3.084240e-06,3.603419e-06,-5.541900e-06,1.467028e-06,326.974548,402.748444,402.748444,...,-29496.50,0.007562,51.348999,1004.137275,0.000061,0.000029,0.000078,0.000029,0.000107,300.915234
78,12.184855,0.000025,0.000002,-2.187550e-06,2.550591e-06,-5.712461e-06,1.519241e-06,325.978271,402.592041,402.592041,...,-25170.50,0.008089,49.529308,1005.254336,0.000108,0.000007,0.000101,0.000009,0.000110,300.924149
79,13.254792,0.000017,0.000002,-9.022481e-07,1.410654e-06,-4.320523e-06,-2.568056e-07,322.068451,403.182495,403.182495,...,-22269.75,0.011431,51.293259,1005.059487,0.000130,0.000012,0.000156,0.000011,0.000167,300.696243


In [12]:
#For vars with 21 vertical levels vorticity humidity file##################
TCp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2 
    
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['vort','rhum']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCp_ts.append(ts_pdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [13]:
dm1=xr.open_dataset(datapath+'/gpot/gpot_2020.nc')

In [14]:
#geopotential
TCgp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2 
    
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['gpot']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCgp_ts.append(ts_pdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [15]:
dm1=xr.open_dataset(datapath+'/vvel/w_wnd_2020.nc')

In [16]:
#Vertical velocity
TCvv_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2 
    
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['vvel']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCvv_ts.append(ts_pdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [17]:
preslv = [str(int(obj)) for obj in dm1.plev.data/100]

In [18]:
dx=xr.open_dataset(datapath+'/tlevs/tlevs_2020.nc')

In [19]:
plev=[str(int(obj)) for obj in dx.plev.data/100]

In [20]:
#storm1

#storm1 vongfong

maindf_vongfong=pd.read_csv(target+'2020_w10max_nwp_vongfong.csv',delimiter=r",")
maindf_vongfong.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_vongfong=maindf_vongfong.drop('a', axis=1)
extra=pd.read_csv(output+'nwp/inder_2020_nwp_vongfong.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)
rlev_vongfong=pd.DataFrame.from_dict(TCrl_ts[0])
prlev_vongfong=pd.DataFrame.from_dict(TCpr_ts[0])
slev_vongfong=pd.DataFrame.from_dict(TCsl_ts[0])
prlev_vongfong['mslp']=prlev_vongfong['mslp']/100

ts1_vongfong=pd.concat([maindf_vongfong,extra,slev_vongfong,rlev_vongfong,prlev_vongfong], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[0]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[0]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[0]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[0]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}
plev_vongfong=pd.DataFrame.from_dict(alldict)
ts_vongfong=pd.concat([ts1_vongfong,plev_vongfong], axis=1, join='inner')

In [21]:
ts_vongfong

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,10.309203,-0.000001,-0.000005,5.563559e-06,7.814137e-06,5.854761e-06,-6.114945e-06,336.610168,401.203491,401.203491,...,-0.019492,-0.051044,-0.091626,-0.094532,-0.042651,-0.035141,-0.060328,-0.054461,-0.025410,-0.013855
1,10.910505,0.000008,-0.000003,-3.635838e-06,-9.401367e-07,7.783531e-07,4.811327e-06,337.293274,400.864471,400.864471,...,-0.039350,-0.029094,-0.018725,-0.061797,-0.075534,-0.070321,-0.071761,-0.043653,-0.004523,0.010754
2,9.938855,0.000017,0.000018,1.688236e-06,-3.315896e-06,-8.495769e-06,2.562160e-06,336.432892,399.526886,399.526886,...,-0.149567,-0.139949,-0.074354,-0.069641,-0.036546,-0.015965,-0.055560,-0.051818,-0.010704,0.009013
3,8.958860,0.000019,0.000022,6.507719e-07,-4.473000e-06,5.714251e-07,6.604291e-06,335.481506,399.772156,399.772156,...,-0.161566,-0.148302,-0.124128,-0.191418,-0.171801,-0.154533,-0.159009,-0.121340,-0.070170,-0.048630
4,9.296708,0.000013,0.000016,-7.199931e-06,-6.999429e-06,6.906871e-06,7.085467e-06,336.512817,399.554321,399.554321,...,-0.120516,-0.082945,-0.075441,-0.169412,-0.151464,-0.128848,-0.142185,-0.094509,-0.047671,-0.024654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,12.185334,0.000024,0.000012,8.853050e-06,3.857888e-06,-1.038136e-05,-1.042617e-05,328.694977,402.119659,402.119659,...,-0.165234,-0.198074,-0.163438,-0.090571,0.056812,0.021010,-0.113343,-0.074103,-0.020460,-0.000033
77,12.407811,0.000016,-0.000004,3.084240e-06,3.603419e-06,-5.541900e-06,1.467028e-06,326.974548,402.748444,402.748444,...,-0.028604,-0.052018,-0.030457,-0.031982,-0.003980,-0.022830,-0.080406,-0.034010,0.004413,0.011782
78,12.184855,0.000025,0.000002,-2.187550e-06,2.550591e-06,-5.712461e-06,1.519241e-06,325.978271,402.592041,402.592041,...,-0.073188,-0.075346,-0.043746,-0.037282,-0.059837,-0.066082,-0.069479,-0.055125,-0.018088,0.004166
79,13.254792,0.000017,0.000002,-9.022481e-07,1.410654e-06,-4.320523e-06,-2.568056e-07,322.068451,403.182495,403.182495,...,-0.030108,-0.034585,0.008081,0.025719,-0.031590,-0.105950,-0.120141,-0.079857,-0.021972,-0.000440


In [22]:
#storm 4 chanhom

maindf_chanhom=pd.read_csv(target+'2020_w10max_nwp_chanhom.csv',delimiter=r",")
maindf_chanhom.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_chanhom=maindf_chanhom.drop('a', axis=1)
extra=pd.read_csv(output+'nwp/inder_2020_nwp_chanhom.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)
rlev_chanhom=pd.DataFrame.from_dict(TCrl_ts[1])
prlev_chanhom=pd.DataFrame.from_dict(TCpr_ts[1])
slev_chanhom=pd.DataFrame.from_dict(TCsl_ts[1])
prlev_chanhom['mslp']=prlev_chanhom['mslp']/100

ts1_chanhom=pd.concat([maindf_chanhom,extra,slev_chanhom,rlev_chanhom,prlev_chanhom], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[1]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[1]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[1]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[1]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}
plev_chanhom=pd.DataFrame.from_dict(alldict)
ts_chanhom=pd.concat([ts1_chanhom,plev_chanhom], axis=1, join='inner')

In [23]:
ts_chanhom

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,9.376366,2.092879e-05,0.000003,-9.492625e-07,1.596399e-07,-1.751216e-06,-3.277715e-06,338.670441,354.683838,352.319794,...,-0.218878,-0.220119,-0.213809,-0.204460,-0.177251,-0.172170,-0.130899,-0.080524,-0.038614,-0.018023
1,9.295533,1.044931e-05,0.000010,9.458436e-06,-4.367001e-06,-3.945375e-06,1.877123e-06,339.252808,354.368103,352.406982,...,-0.230517,-0.243211,-0.179813,-0.171782,-0.168298,-0.150948,-0.077980,-0.032441,0.001946,0.017313
2,10.131128,1.641898e-05,0.000015,7.985401e-06,-5.679445e-06,-5.566510e-06,2.914433e-06,338.594604,354.061829,352.587189,...,-0.253631,-0.257827,-0.182666,-0.176414,-0.233694,-0.238475,-0.118915,-0.050981,-0.007187,0.012778
3,10.922950,1.843215e-05,0.000016,3.257839e-06,-2.021105e-06,7.119737e-07,4.734732e-06,338.228210,353.762207,352.451965,...,-0.241725,-0.251304,-0.235092,-0.270356,-0.304945,-0.270503,-0.143708,-0.083676,-0.042705,-0.022141
4,10.921099,1.746406e-05,0.000015,-3.864217e-06,-1.012786e-05,2.474067e-06,2.514803e-06,338.604980,353.883606,352.550842,...,-0.200024,-0.159393,-0.118041,-0.165288,-0.172795,-0.145339,-0.088106,-0.051658,-0.022557,-0.006776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,15.260516,6.882865e-06,0.000001,6.311119e-06,4.303548e-06,-8.145852e-06,-1.798526e-08,307.860107,349.829712,343.740448,...,-0.020674,-0.043032,0.018042,0.058398,0.034249,-0.056063,-0.072675,-0.061888,-0.033960,-0.010460
106,14.735302,1.320104e-07,-0.000006,-2.592002e-06,1.388817e-06,4.577279e-06,3.143935e-06,308.141876,349.570618,344.212708,...,0.065259,0.069102,0.036965,-0.004649,-0.045241,-0.070439,-0.115235,-0.119764,-0.049286,0.000049
107,14.472641,8.215622e-06,-0.000002,-3.117607e-07,6.945216e-06,-3.222564e-07,-7.365141e-07,310.461548,349.037231,344.887146,...,0.006628,-0.017471,-0.024171,-0.043218,-0.037979,0.010520,-0.013746,-0.068106,-0.042926,-0.009974
108,13.135821,-8.778028e-07,0.000008,-3.190881e-06,2.965728e-08,-3.890142e-06,-2.785965e-06,308.148254,348.835663,344.156189,...,-0.025623,-0.021338,0.041293,0.054789,0.065842,0.058151,-0.062695,-0.080582,-0.040554,-0.009860


In [24]:
ts_vongfong.to_csv(output+'inner/inner_wp_all_vongfong.csv')
ts_chanhom.to_csv(output+'inner/inner_wp_all_chanhom.csv')

In [91]:
#storm 3 saudel

maindf_saudel=pd.read_csv(target+'2020_w10max_nwp_saudel.csv',delimiter=r",")
maindf_saudel.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_saudel=maindf_saudel.drop('a', axis=1)
extra=pd.read_csv(output+'nwp/inder_2020_nwp_saudel.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)


In [94]:
rlev_saudel=pd.DataFrame.from_dict(TCrl_ts[2])
prlev_saudel=pd.DataFrame.from_dict(TCpr_ts[2])
slev_saudel=pd.DataFrame.from_dict(TCsl_ts[2])
prlev_saudel['mslp']=prlev_saudel['mslp']/100

In [95]:
ts1_saudel=pd.concat([maindf_saudel,extra,slev_saudel,rlev_saudel,prlev_saudel], axis=1, join='inner')

In [96]:
tempvortdict = {'vort_'+plev[i]:TCp_ts[2]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[2]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[2]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[2]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

In [97]:
plev_saudel=pd.DataFrame.from_dict(alldict)
ts_saudel=pd.concat([ts1_saudel,plev_saudel], axis=1, join='inner')

In [99]:
ts_saudel.to_csv(output+'inner/inner_wp_all_saudel.csv')

In [100]:
#storm 4 molave

maindf_molave=pd.read_csv(target+'2020_w10max_nwp_molave.csv',delimiter=r",")
maindf_molave.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_molave=maindf_molave.drop('a', axis=1)
extra=pd.read_csv(output+'nwp/inder_2020_nwp_molave.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)
rlev_molave=pd.DataFrame.from_dict(TCrl_ts[3])
prlev_molave=pd.DataFrame.from_dict(TCpr_ts[3])
slev_molave=pd.DataFrame.from_dict(TCsl_ts[3])
prlev_molave['mslp']=prlev_molave['mslp']/100

ts1_molave=pd.concat([maindf_molave,extra,slev_molave,rlev_molave,prlev_molave], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[3]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[3]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[3]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[3]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}
plev_molave=pd.DataFrame.from_dict(alldict)
ts_molave=pd.concat([ts1_molave,plev_molave], axis=1, join='inner')

In [102]:
ts_molave.to_csv(output+'inner/inner_wp_all_molave.csv')

In [103]:
#storm 5 goni
maindf_goni=pd.read_csv(target+'2020_w10max_nwp_goni.csv',delimiter=r",")
maindf_goni.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_goni=maindf_goni.drop('a', axis=1)
extra=pd.read_csv(output+'nwp/inder_2020_nwp_goni.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)
rlev_goni=pd.DataFrame.from_dict(TCrl_ts[4])
prlev_goni=pd.DataFrame.from_dict(TCpr_ts[4])
slev_goni=pd.DataFrame.from_dict(TCsl_ts[4])
prlev_goni['mslp']=prlev_goni['mslp']/100

ts1_goni=pd.concat([maindf_goni,extra,slev_goni,rlev_goni,prlev_goni], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[4]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[4]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[4]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[4]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}
plev_goni=pd.DataFrame.from_dict(alldict)
ts_goni=pd.concat([ts1_goni,plev_goni], axis=1, join='inner')
ts_goni

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,8.367717,0.000030,1.590303e-05,7.356431e-06,1.214419e-06,5.962330e-07,-1.424135e-06,331.848175,351.363892,348.766205,...,-0.269152,-0.298407,-0.282164,-0.285947,-0.224323,-0.154606,-0.073686,-0.028095,0.000581,0.010498
1,9.449281,0.000030,1.827138e-05,2.173332e-06,5.862377e-07,-1.973572e-06,-8.502179e-07,330.939117,350.243774,349.241089,...,-0.243749,-0.259310,-0.228003,-0.212812,-0.210290,-0.176840,-0.126462,-0.057795,-0.007975,0.012646
2,12.327559,0.000009,2.140960e-05,1.090387e-05,7.192536e-06,-5.287471e-06,-1.468169e-06,333.098633,348.851044,347.741638,...,-0.159618,-0.208045,-0.211802,-0.183534,-0.149399,-0.114429,-0.102378,-0.067866,-0.022048,0.000943
3,12.489228,0.000018,2.562352e-05,1.746218e-05,1.149249e-05,-1.743597e-06,3.527325e-07,331.694092,349.465851,347.718384,...,-0.301193,-0.376839,-0.407216,-0.409952,-0.376526,-0.297281,-0.217049,-0.143700,-0.062510,-0.023518
4,11.228815,0.000037,2.299392e-05,2.092692e-05,6.591619e-06,-1.336298e-07,-7.195069e-07,333.489990,349.515747,347.472534,...,-0.303415,-0.376833,-0.391419,-0.393253,-0.334075,-0.212005,-0.116779,-0.075655,-0.033486,-0.010773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,12.553186,0.000015,-1.735051e-06,-1.315080e-05,-1.676779e-05,-4.615766e-06,8.291820e-06,330.858582,350.205811,348.100128,...,-0.053455,0.034132,0.126938,0.086644,-0.004293,-0.181541,-0.413227,-0.278603,-0.121440,-0.037766
93,12.279484,0.000018,2.420765e-06,-8.579384e-06,-1.973723e-05,1.458764e-06,1.517077e-05,330.239532,349.540497,347.643982,...,-0.075256,0.011803,0.094151,-0.036279,-0.099763,-0.220002,-0.419695,-0.262588,-0.086795,-0.002135
94,10.930732,0.000007,-4.595220e-07,1.057944e-07,-7.029741e-06,3.092277e-06,3.905538e-06,324.859802,350.248230,348.099854,...,-0.024639,-0.004127,0.006034,-0.066011,-0.033516,-0.110735,-0.303484,-0.202085,-0.072954,-0.010217
95,11.166134,0.000002,-6.647833e-06,-3.899162e-06,-6.179701e-06,8.060028e-06,4.362748e-06,320.559174,350.367462,348.103394,...,0.049856,0.076343,0.058157,-0.039621,-0.065429,-0.142826,-0.289336,-0.216602,-0.110136,-0.060735


In [104]:
ts_goni.to_csv(output+'inner/inner_wp_all_goni.csv')

In [105]:
#storm 6 atsani
maindf_atsani=pd.read_csv(target+'2020_w10max_nwp_atsani.csv',delimiter=r",")
maindf_atsani.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_atsani=maindf_atsani.drop('a', axis=1)
extra=pd.read_csv(output+'nwp/inder_2020_nwp_atsani.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)
rlev_atsani=pd.DataFrame.from_dict(TCrl_ts[5])
prlev_atsani=pd.DataFrame.from_dict(TCpr_ts[5])
slev_atsani=pd.DataFrame.from_dict(TCsl_ts[5])
prlev_atsani['mslp']=prlev_atsani['mslp']/100

ts1_atsani=pd.concat([maindf_atsani,extra,slev_atsani,rlev_atsani,prlev_atsani], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[5]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[5]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[5]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[5]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}
plev_atsani=pd.DataFrame.from_dict(alldict)
ts_atsani=pd.concat([ts1_atsani,plev_atsani], axis=1, join='inner')
ts_atsani

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,13.654966,3.250962e-05,8.415326e-06,-1.079934e-05,-1.733789e-06,0.000004,-5.103514e-06,337.551910,350.967773,349.370361,...,-0.157595,-0.102186,-0.162688,-0.199878,-0.102553,0.031970,0.102614,0.073818,0.015571,-0.002955
1,13.333771,3.800994e-05,6.226319e-06,-1.114594e-05,3.774389e-06,0.000008,-1.716448e-06,338.075439,351.065887,349.038391,...,-0.189562,-0.170216,-0.243111,-0.317948,-0.236609,-0.099976,-0.082233,-0.063325,-0.046232,-0.029405
2,13.895884,4.568494e-05,1.353166e-05,-5.222186e-07,7.107878e-06,0.000010,-7.376450e-06,336.909515,351.134125,348.775146,...,-0.321718,-0.345572,-0.405233,-0.463608,-0.319323,-0.196298,-0.115407,-0.074188,-0.024602,0.003886
3,14.529068,5.294114e-05,1.938432e-05,1.272257e-05,1.179825e-05,0.000006,-1.222570e-05,335.190125,351.260529,349.172760,...,-0.364717,-0.426834,-0.500615,-0.507501,-0.338421,-0.277703,-0.191371,-0.100442,-0.027442,0.009256
4,14.130989,4.391254e-05,2.829460e-05,1.903524e-05,4.756615e-06,-0.000016,-2.586564e-06,334.479858,351.155609,349.602264,...,-0.399424,-0.464807,-0.387073,-0.311642,-0.246776,-0.157137,-0.132155,-0.113099,-0.054234,-0.014204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,14.715964,4.517028e-07,-2.781457e-06,-1.505086e-06,3.793326e-06,0.000010,-2.253578e-06,321.156982,352.486633,350.058655,...,0.014101,0.011094,-0.040007,-0.083310,-0.104310,-0.186706,-0.160513,-0.157613,-0.105195,-0.076619
70,15.020427,-2.634104e-06,3.636680e-06,-8.155421e-07,3.587869e-06,0.000008,3.420132e-08,318.570557,352.428070,349.911011,...,0.013977,0.006069,-0.025150,-0.064229,-0.096511,-0.185668,-0.163823,-0.132158,-0.077781,-0.047235
71,14.803987,-3.647683e-06,8.870117e-06,-5.950974e-07,-2.493488e-06,0.000004,2.765601e-06,315.623291,352.288818,349.610626,...,-0.011028,-0.003204,0.030447,0.001787,-0.045317,-0.132210,-0.128933,-0.073452,-0.013819,0.013334
72,13.906894,-2.137216e-06,5.670727e-07,3.796495e-06,-6.555103e-08,-0.000002,-3.116218e-08,314.585815,352.522705,349.666351,...,0.011295,0.001560,0.063092,0.077198,0.058105,-0.024990,-0.063047,-0.029642,0.009193,0.005643


In [106]:
ts_atsani.to_csv(output+'inner/inner_wp_all_atsani.csv')

In [108]:
#storm 7 vamco
maindf_vamco=pd.read_csv(target+'2020_w10max_nwp_vamco.csv',delimiter=r",")
maindf_vamco.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_vamco=maindf_vamco.drop('a', axis=1)
extra=pd.read_csv(output+'nwp/inder_2020_nwp_vamco.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)
rlev_vamco=pd.DataFrame.from_dict(TCrl_ts[6])
prlev_vamco=pd.DataFrame.from_dict(TCpr_ts[6])
slev_vamco=pd.DataFrame.from_dict(TCsl_ts[6])
prlev_vamco['mslp']=prlev_vamco['mslp']/100

ts1_vamco=pd.concat([maindf_vamco,extra,slev_vamco,rlev_vamco,prlev_vamco], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[6]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[6]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[6]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[6]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}
plev_vamco=pd.DataFrame.from_dict(alldict)
ts_vamco=pd.concat([ts1_vamco,plev_vamco], axis=1, join='inner')
ts_vamco

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,12.301943,0.000034,3.935914e-05,2.301252e-05,-0.000003,-0.000007,-0.000008,335.176880,351.065857,349.487091,...,-0.429923,-0.476252,-0.360352,-0.317766,-0.245955,-0.243994,-0.193995,-0.107232,-0.032387,0.000495
1,12.164107,0.000030,3.894729e-05,3.033575e-05,0.000009,-0.000004,0.000002,335.679382,351.425781,350.216553,...,-0.506310,-0.608837,-0.579762,-0.593913,-0.616277,-0.623864,-0.439612,-0.257455,-0.113403,-0.043453
2,11.694481,0.000037,4.089281e-05,3.292633e-05,0.000012,-0.000004,-0.000004,335.140381,352.164673,351.181244,...,-0.593365,-0.719954,-0.713271,-0.684060,-0.623321,-0.553757,-0.360561,-0.201441,-0.075248,-0.013993
3,12.752129,0.000044,2.951113e-05,1.533757e-05,0.000003,0.000005,-0.000002,335.450409,351.865417,350.744324,...,-0.513160,-0.562215,-0.557060,-0.621180,-0.543813,-0.425319,-0.264064,-0.154761,-0.056961,-0.010122
4,12.502798,0.000042,3.501577e-05,3.285955e-05,0.000022,0.000015,-0.000008,335.887512,351.208984,350.723450,...,-0.478959,-0.613283,-0.805208,-0.849901,-0.743551,-0.581576,-0.289776,-0.170742,-0.077492,-0.029195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,13.873990,-0.000009,-4.118349e-07,6.354632e-06,0.000002,0.000003,0.000004,307.785950,352.645599,349.538696,...,0.009330,0.006360,-0.032430,-0.065741,-0.167161,-0.267635,-0.232226,-0.045639,0.000588,0.011707
63,14.389834,-0.000012,1.114982e-06,5.508081e-06,0.000002,0.000004,0.000006,303.966522,352.751221,349.099121,...,0.013619,0.008235,-0.055016,-0.096038,-0.236834,-0.319392,-0.156961,0.021870,0.043630,0.040154
64,13.112446,-0.000010,-1.615069e-06,-3.792945e-07,-0.000002,0.000005,0.000010,300.972595,353.047821,349.021545,...,0.027735,0.045319,0.002998,-0.092992,-0.169909,-0.187990,-0.043701,0.050874,0.052986,0.044466
65,12.451212,-0.000010,-3.569903e-06,-4.802502e-07,-0.000006,0.000005,0.000008,298.801605,353.123169,348.590576,...,0.034259,0.056874,0.035741,-0.060762,-0.093383,-0.118826,-0.037434,0.030718,0.023796,0.017375


In [109]:
ts_vamco.to_csv(output+'inner/inner_wp_all_vamco.csv')